In [2]:
import pandas as pa
import numpy as nm
from keras.datasets import imdb
from keras.layers import Embedding, Dense, Dropout, LSTM, Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

In [3]:
(xTrain, yTrain), (xTest, yTest) = imdb.load_data()
indexes = imdb.get_word_index()

In [4]:
print(len(indexes))

88584


In [5]:
# Duygu analizi için fazla kelîme var.
# Önişlemede kelîmelerin süzgeçten geçirilip, geçirilmediğine
# bakılabilir; bunun için birkaç deneme yapabiliriz:
print(f"0 rakamının indeksi: {indexes['0']}")
print(f"'the' ilgisiz kelîmesinin indeksi: {indexes['the']}")
print(f"'a' ilgisiz kelîmesinin indeksi: {indexes['a']}")
print(f"'I' ilgisiz zamirinin indeksi: {indexes['your']}")

0 rakamının indeksi: 2238
'the' ilgisiz kelîmesinin indeksi: 1
'a' ilgisiz kelîmesinin indeksi: 3
'I' ilgisiz zamirinin indeksi: 126


In [6]:
# Duygu analizinde genellikle kelîmelerin sırası önemli değildir.
# Çünkü biz, ne yorumun anlamını anlamaya çalışıyor,
# ne çeviri yapmaya çalışıyor; ne de bahsettiği konuyla ilgileniyoruz;
# Biz sadece olumlu yorumlarda saptanan desenleri ve olumsuz yorumlarda
# saptanan desenleri anlamaya çalışıyoruz.
# Bu, genellikle kelîme frekansı gibi basit analizle çözülebilir.
# Dikkat, kelîmelerin sırasının öneminin olmaması, kelîmelerin
# yinelenen sinir ağında işlenmesine gerek olmadığı anlamına gelmez.
# Çünkü, kelîmelerin az da olsa bağlam bilgisinin çıkarılması,
# model performansı için önemlidir.

# Bu proje için;
# En çok kullanılan 15000 kelîme üzerinde duygu analizi
# yapmaya çalışıp, yetersiz olunması durumunda veri seti üzerinde
# temizlik yapmaya girişilebilir:

# En çok kullanılan 15000 kelîmeyi almak istersek;
(xTrain, yTrain), (xTest, yTest) = imdb.load_data(num_words = 15000)
print(f"0 rakamının indeksi: {indexes['0']}")
print(f"'the' ilgisiz kelîmesinin indeksi: {indexes['the']}")
print(f"'a' ilgisiz kelîmesinin indeksi: {indexes['a']}")
print(f"'I' ilgisiz zamirinin indeksi: {indexes['your']}")
# Durma kelîmeleri hâlen duruyor.

0 rakamının indeksi: 2238
'the' ilgisiz kelîmesinin indeksi: 1
'a' ilgisiz kelîmesinin indeksi: 3
'I' ilgisiz zamirinin indeksi: 126


In [7]:
# Kelîmelerin eşit uzunluğa getirilmesi gerekiyor:
maxLen = 100# Cümle uzunluğu
xTrain = pad_sequences(xTrain, maxlen = maxLen)
xTest = pad_sequences(xTest, maxlen = maxLen)

In [8]:
# Katmanların, model çağrılarının ve modelin (Sequential API ile) oluşturulması

model = Sequential()
model.add(Input(shape=(maxLen,)))
model.add(Embedding(
    input_dim = 15000, output_dim = 150, input_length = maxLen
    ))
model.add(LSTM(64, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

callEarlyStopping = EarlyStopping(monitor = 'val_loss', patience = 3,
                                  restore_best_weights = True)

filePath = "C:\AIMLProjects\Keras\imdbSentimentAnalysis\model.h5"
callModelCheckpoint = ModelCheckpoint(filePath, save_best_only = True)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 150)            │       2,250,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          55,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,306,097 (8.80 MB)

 Trainable params: 2,306,097 (8.80 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Modelin derlenmesi ve eğitim:

model.compile(loss = 'binary_crossentropy', optimizer = 'adam',
              metrics = ['accuracy'])

model.fit(x = xTrain, y = yTrain, epochs = 15, batch_size = 256,
          validation_split = 0.2,
          callbacks = [callEarlyStopping, callModelCheckpoint]
          )


res = model.history.history

print(f"Doğruluk : {res['accuracy'][-1]}")# 0.8217499852180481
print(f"Kayıp : {res['loss'][-1]}")# 0.4583764672279358
print(f"Doğrulama seti doğruluğu : {res['val_accuracy'][-1]}")# 0.6930000185966492
print(f"Doğrulama kaybı : {res['val_loss'][-1]}")# 0.5820649862289429

Epoch 1/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 742ms/step - accuracy: 0.5936 - loss: 280.0533

79/79 ━━━━━━━━━━━━━━━━━━━━ 73s 824ms/step - accuracy: 0.5943 - loss: 284.4839 - val_accuracy: 0.6916 - val_loss: 24.3266
Epoch 2/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 60s 761ms/step - accuracy: 0.7106 - loss: 70862032.0000 - val_accuracy: 0.5840 - val_loss: 6220899.5000
Epoch 3/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 56s 710ms/step - accuracy: 0.5746 - loss: 895161.8125 - val_accuracy: 0.5914 - val_loss: 2794.2463
Epoch 4/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 59s 747ms/step - accuracy: 0.6585 - loss: 2637.3892 - val_accuracy: 0.6856 - val_loss: 1724.3610
Doğruluk : 0.6909999847412109
Kayıp : 2109.111572265625
Doğrulama seti doğruluğu : 0.6855999827384949
Doğrulama kaybı : 1724.3609619140625


In [10]:
testLoss, testAccuracy = model.evaluate(xTest, yTest, batch_size = 256)

print(f"Test kaybı: {testLoss}")# 0.5038241147994995
print(f"Test Doğruluğu: {testAccuracy}")# 0.7570800185203552

# Görüldüğüz üzere başarı %~70
# ve böylesine basit bir görev için bu çok düşük!..

# Sorun ne??
# Sorun, durma kelîmelerinin sürekli geçmesinden ötürü
# 88584 kelîme içerisinden en çok geçen 15 bin kelîmeyi aldığımızda
# pek çok durma kelîmesini alıyor oluşumuzdur.
# Durma kelîmeleri ('the', 'a', 'an' gibi) hem olumlu, hem de
# olumsuz yorumlarda geçtiğinden model başarısı yükselmiyor
# Ayrıca modelin aşırı uyumunu engellemek için hiç 'Dropout'
# katmanı eklemedik; bu kadar fazla parametreye 'Dropout' eklenmeden olmaz.

98/98 ━━━━━━━━━━━━━━━━━━━━ 25s 251ms/step - accuracy: 0.6932 - loss: 932790.0000
Test kaybı: 3754645.5
Test Doğruluğu: 0.6926000118255615


In [96]:
# Yeni verilerle çalışmadan evvel hâfızayı temizlemek münâsip olur:
del xTrain, xTest, yTrain, yTest

In [74]:
# Çözüm önerisi:

# Her kelîmenin indeksini bildiğimize göre kelîmeleri
# metîn olarak alıp, üzerinde ön işleme yapabiliriz:

# Her sayının yerine kelîmeyi koymak yerine
# Veri setinin ham hâlini indirmek daha uygun olur.
# Veri setinin asıl adresi şudur:
#r"https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
# Fakat arşivden çıkarma gibi faaliyetlerden sıyrılmak için
# pandas ile verilen adresten verinin çekildiği fonksiyonu kullanabiliriz
# Bu verisi HuggingFace ile sunuluyor:
# r"hf://datasets/scikit-learn/imdb/IMDB Dataset.csv"
# Bunun için huggingface_hub ve fsspec kütüphânelerini kurmak lazım:
# pip install huggingface_hub, fsspec
# Daha fazla bilgi ve atıf burada mevcut: https://huggingface.co/datasets/scikit-learn/imdb
dataURL = r"hf://datasets/scikit-learn/imdb/IMDB Dataset.csv"

import pandas as pa
dfData = pa.read_csv(dataURL)
dfOriginal = dfData.copy()
# İlk yorumlara bakmak istersek:
#                                               review sentiment
# 0  One of the other reviewers has mentioned that ...  positive
# 1  A wonderful little production. <br /><br />The...  positive
# 2  I thought this was a wonderful way to spend ti...  positive
# 3  Basically there's a family where a little boy ...  negative
# 4  Petter Mattei's "Love in the Time of Money" is...  positive

# Veri ciddî şekilde bozuk!!

In [75]:
# Veri ön işleme ve temizlik (durma kelîmelerin dışındakiler):
import re
import string

# Şu uyarıları bir kapatmak lazım:
import warnings
warnings.filterwarnings('ignore')

# Tüm kelîmeleri küçük harfe çevir:
dfData["review"] = [i.lower() for i in dfData["review"]]

# Bağlantı adreslerini çıkart:
patternURL = r"(http[s]?://\S+)|(www.\S+)"
dfData['review'].replace(patternURL, '', regex = True, inplace = True)

# E - posta adreslerinin çıkartılması:
patternEmail = r"\S+@\S+"
dfData['review'].replace(patternEmail, '', regex = True, inplace = True)

# Târih ve saatlerin çıkartılması: 1st.05.2024, 01.05.2024, 1.5.2024, 1.5.24
# bunların diğer kombinasyonları ve # nokta (.) yerine '-' ile kullanılan versiyonları
patternDate = r"\d{1,2}(th|st|rd|nd)?[\.-/]\d{1,2}[\.-/]\d{2,4}"
dfData['review'].replace(patternDate, '', regex = True, inplace = True)

# HTML ve XML gibi etiket betik dillerinin kodlarının çıkartılması:
# Ayrac işâretinin varlığına ve mevkîsine göre üç çeşit var : <br/> <html> </html>
# Fakat veri setinde arada boşluk olduğundan <br /> ifâdesi düzenli ifâde kapsamında olmalı:
patternScript = r"<[/]?\S+[ ]?[/]?>"
dfData['review'].replace(patternScript, '', regex = True, inplace = True)

# Noktalama işâretlerinin çıkartılması:
# puncs = "[!\"#$%&'()*+,-./:;<=>?@^_{|}~9}]`"
dfData['review'] = [i.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))) for i in dfData['review']]

# Metnin kelîmelere ayrılması ve boşlukların kaldırılması:
dfData['review'] = [i.split() for i in dfData['review']]

In [77]:
# Veri ön işleme, devâmı (durma kelîmelerinin çıkarılması)
# Durma kelîmelerinin çıkartılması:
import nltk# Eğer yüklemediyseniz 'pip install nltk' kodu ile yükleyiniz.
from nltk.corpus import stopwords
nltk.download('stopwords')

# Durma kelîmelerini çıkarmadan evvel, iki metnin uzunluğuna bakmak istersek;
# Dikkat, bu işlem uzun sürmektedir; bu işlem yerine buradaki her bir
# kelîmeyi aralarına veyâ (|) işâreti koyarak bir düzenli ifâde hâline
# getirmek performansı arttırabilir.
firstSentenceLen = len(dfData['review'][0])
secondSentenceLen = len(dfData['review'][1])
print(f"İlk cümlenin uzunluğu (kelîme sayısı olarak): {firstSentenceLen}")
print(f"İkinci cümlenin uzunluğu (kelîme sayısı olarak): {secondSentenceLen}")
# Temizlenmiş ilk cümlenin uzunluğu (kelîme sayısı olarak): 307
# Temizlenmiş ikinci cümlenin uzunluğu (kelîme sayısı olarak): 162
dfData['review'] = [[i for i in sent if i not in stopwords.words('english')] for sent in dfData['review']]

firstSentenceLen = len(dfData['review'][0])
secondSentenceLen = len(dfData['review'][1])
print(f"Temizlenmiş ilk cümlenin uzunluğu (kelîme sayısı olarak): {firstSentenceLen}")
print(f"Temizlenmiş ikinci cümlenin uzunluğu (kelîme sayısı olarak): {secondSentenceLen}")
# Temizlenmiş ilk cümlenin uzunluğu (kelîme sayısı olarak): 171
# Temizlenmiş ikinci cümlenin uzunluğu (kelîme sayısı olarak): 90

# Görüldüğü üzere ilk cümlenin neredeyse yarısı anlamsız kelîmelerden
# oluşuyormuş; ikinci cümlede ise 72 adet anlamsız kelîmeden kurtulmuş olduk
# Bu büyük veri temizliğinden sonra kelîmelerin köklerini bulmaya başlanabilir.

# Veriyi kaybetmemek için kaydedebilirsiniz:
dfData.to_json('cleaned_IMDB.json')

[nltk_data] Downloading package stopwords to C:\Users\Yazılım
[nltk_data]     alanı\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


İlk cümlenin uzunluğu (kelîme sayısı olarak): 314
İkinci cümlenin uzunluğu (kelîme sayısı olarak): 160
Temizlenmiş ilk cümlenin uzunluğu (kelîme sayısı olarak): 163
Temizlenmiş ikinci cümlenin uzunluğu (kelîme sayısı olarak): 86


In [78]:
# Görüldüğü üzere ilk cümlenin neredeyse yarısı anlamsız kelîmelerden
# oluşuyormuş; ikinci cümlede ise 72 adet anlamsız kelîmeden kurtulmuş olduk
# Bu büyük veri temizliğinden sonra kelîmelerin köklerini bulma işlemine başlanabilir.

# Kelîme gövdelerinin elde edilmesi:
# Kelîme gövdelerinin elde edilmesi için yardımcı kütüphâneye ihtiyaç var
# nltk kütüphânesi bu işlem için destek veriyor..
from nltk.stem import PorterStemmer
prStem = PorterStemmer()
dfData['review'] = [[prStem.stem(word) for word in sentence] for sentence in dfData['review']]

In [79]:
# İlk örnek üzerinden verinin dönüşümünü incelemek istersek;
print(f"Asıl veri:\n{dfOriginal['review'][0]}\n\nTemizlenmiş yorum:\n{" ".join(dfData['review'][0])}")

Asıl veri:
One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due

In [521]:
# Veri gerçekten kısalmış; fakat istemediğimiz bir şey 'reviewers' gibi hem yapım, hem çekim eki almış kelîmelerden
# yapım eki de çıkarılmış; bunu yapmadığımız zamân model başarısı daha iyi olabilir;
# Eğer iyi bir sonuç alamazsak, metîn gövdeleme işlemini yapmadan deneyebiliriz.
# Bâzı durumlarda kelîmenin kökünü almak yerine gövdesini almak daha iyi bir çözüm olabilir,
# Veriyi kelîme köklerini almadan evvel kaydettiğimiz için kelîme kökleme yerine kelîme gövdeleme işlemi uygulanıp,
# performans karşılaştırması yapılabilir.
# Bu arada PorterStemmer, olumsuzlukla ilgili yapım eklerini çıkartmıyor:
print(f"'deactivate' kelîmesnin gövdesi: {prStem.stem('deactivate')}")
print(f"'unsuccessful' kelîmesnin gövdesi: {prStem.stem('unsuccessful')}")
# Devâm etmeden evvel köklenmiş veriyi ayrı dosyada saklamak isteyebiliriz:
dfData.to_json('cleaned_and_stemmed_IMDB.json')

'deactivate' kelîmesnin gövdesi: deactiv
'unsuccessful' kelîmesnin gövdesi: unsuccess


In [97]:
# Şimdi kelîmeleri birleştirip;
dfData['review'] = [' '.join(sent) for sent in dfData['review']]

In [101]:
# vektör hâline getirmek gerekiyor:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 15000)

In [107]:
# Verileri sayı dizisi şekline getirmek için;
tokenizer.fit_on_texts(dfData['review'])

In [113]:
dfData['sequences'] = tokenizer.texts_to_sequences(dfData['review'])

In [221]:
# Metînleri eşit uzunluğa getirmek gerekiyor.
maxLength = 100
dfData['readySeq'] = list(pad_sequences(dfData['sequences'], maxlen = maxLength))

In [237]:
# Veri setini test ve eğitim olarak ikiye ayırmadan evvel, y değerlerini sayısala çevirmeliyiz:
dfData['readySentiment'] = dfData['sentiment'].apply(lambda x: nm.uint8(1) if x == 'positive' else nm.uint8(0))

In [376]:
# Verileri ayırmamız lazım:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(dfData['readySeq'], dfData['readySentiment'],test_size = 0.3, random_state = 1)

In [379]:
# Verinin iç boyut biçimi şu an doğru görünmediği için önce veri düzleştirilmeli
xTrain = nm.array([i.flatten() for i in xTrain])
xTest = nm.array([i.flatten() for i in xTest])

In [417]:
# Etiket verileri numpy dizisi hâline getirilmeli
yTrain = yTrain.to_numpy()
yTest = yTest.to_numpy()

In [537]:
# Yeni model oluşturmalı ve bu sefer Dropout katmanı da eklemeliyiz:
# Bu tür durumlar için modeli Sequential API yerine fonksiyonel API yöntemiyle oluşturmak daha iyidir.
model = Sequential()
model.add(Input(shape=(100,)))
model.add(Embedding(
    input_dim = 15000, output_dim = 150, input_length = maxLen
    ))
model.add(LSTM(32, activation = 'relu', return_sequences = True))
model.add(Dropout(rate=0.2))
model.add(LSTM(16, activation = 'relu'))
model.add(Dropout(rate=0.1))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(1, activation = 'sigmoid'))

callEarlyStopping = EarlyStopping(monitor = 'val_loss', patience = 3,
                                  restore_best_weights = True)

filePath = "C:\AIMLProjects\Keras\imdbSentimentAnalysis\model_v2.h5"
callModelCheckpoint = ModelCheckpoint(filePath, save_best_only = True)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 150)            │       2,250,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 100, 32)             │          23,424 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │             544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,277,137 (8.69 MB)

 Trainable params: 2,277,137 (8.69 MB)

 Non-trainable params: 0 (0.00 B)

In [450]:
# Modelin derlenmesi ve eğitim:

model.compile(loss = 'binary_crossentropy', optimizer = 'adam',
              metrics = ['accuracy'])

model.fit(x = xTrain, y = yTrain, epochs = 15, batch_size = 256,
          validation_split = 0.2,
          callbacks = [callEarlyStopping, callModelCheckpoint]
          )


res = model.history.history

print(f"Doğruluk : {res['accuracy'][-1]}")# 
print(f"Kayıp : {res['loss'][-1]}")# 
print(f"Doğrulama seti doğruluğu : {res['val_accuracy'][-1]}")# 
print(f"Doğrulama kaybı : {res['val_loss'][-1]}")# 

Epoch 1/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step - accuracy: 0.6744 - loss: 0.6297

110/110 ━━━━━━━━━━━━━━━━━━━━ 73s 562ms/step - accuracy: 0.6751 - loss: 0.6288 - val_accuracy: 0.8484 - val_loss: 0.3570
Epoch 2/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step - accuracy: 0.8804 - loss: 0.4073

110/110 ━━━━━━━━━━━━━━━━━━━━ 59s 538ms/step - accuracy: 0.8804 - loss: 0.4071 - val_accuracy: 0.8531 - val_loss: 0.3535
Epoch 3/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 59s 538ms/step - accuracy: 0.9105 - loss: 329.0284 - val_accuracy: 0.8243 - val_loss: 0.4276
Epoch 4/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 59s 532ms/step - accuracy: 0.8640 - loss: 0.3692 - val_accuracy: 0.8201 - val_loss: 0.4083
Epoch 5/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 82s 533ms/step - accuracy: 0.8968 - loss: 0.2870 - val_accuracy: 0.8249 - val_loss: 0.4139
Doğruluk : 0.8934285640716553
Kayıp : 0.2831476628780365
Doğrulama seti doğruluğu : 0.8248571157455444
Doğrulama kaybı : 0.4138679504394531


In [451]:
# Oldukça zorlu bir veri setiydi. Bu veri setinin üstesinden gelmek için şunlar yapılabilir:
# - alınan kelîme sayısını biraz daha arttırmak,
# - durma kelîmelerinin kapsamını genişletmek
# - Modele bir iki katman daha eklenebilir (berâberinde Dropout katmanıyla berâber)

# Şu anki hâliyle öncekinden çok daha başarılı bir sistem üretilmiş oldu;
# sistemin biraz ezberlediği görünüyor;
# fakat bundan emîn olmak için test veri seti üzerinde değerlendirme yapmak lazım:
resOfTest = model.evaluate(xTest, yTest, batch_size = 256)

59/59 ━━━━━━━━━━━━━━━━━━━━ 10s 173ms/step - accuracy: 0.8621 - loss: 0.3465


- Modelle ilgili durum hiç de fenâ değil; fakat bu kadar basit bir görev için az görünüyor..
- Bunun önemli sebebinin kelîmeleri gövdelememiz olduğunu düşünüyorum;
- Şimdi elimizdeki temizlenmiş, fakat gövdelenmemiş eğitim veri setiyle deneme yapmak iyi bir fikir olabilir.
- Daha fazla bilgi için Sınıflandırma raporuna ('classification_report') bakılabilir 

- Yazar : Mehmet Akif SOLAK